# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

- What is this dataset about?
    - It is a direct-marketing dataset from a Portuguese bank. Each row is a phone contact to a client, and the target y tells us if the client subscribed to a long-term deposit (yes or no).

- How many campaigns and contacts are represented?
    - There were 17 campaigns between May 2008 and November 2010, with 79,354 total contacts and 6,499 successes, about an 8 percent success rate.

- What predictive task are we solving?
    - Binary classification. Given client and campaign attributes known before or at the time of dialing, predict whether the client will subscribe to the deposit.

- Which algorithms did the paper compare and which performed best?
    - Naive Bayes, Decision Trees, and Support Vector Machines. Support Vector Machines achieved the strongest discrimination by AUC in their best iteration.

- Which evaluation metrics are most relevant here and why?
    - ROC AUC to measure discrimination independent of threshold and class balance. Lift and cumulative Lift are especially useful for marketing because they tell you how many successes you get if you only call the top X percent most likely clients.

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [57]:
import time
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, recall_score, f1_score, precision_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

In [58]:
df = pd.read_csv('data/bank-additional-full.csv', sep = ';')

In [59]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



- This dataset has 20 input features and 1 output target. Most numeric variables (age, campaign, previous, emp.var.rate, etc.) are clean, but pdays uses 999 as a placeholder for “not contacted,” which should be treated as missing or a separate category. Several categorical variables (job, marital, education, default, housing, loan) contain “unknown,” which indicates missing values and should be handled either by imputation or by keeping them as their own category. The variable duration must be dropped, since it is only known after the call and would leak information into the prediction. Finally, categorical variables like job, month, day_of_week, and poutcome will need to be encoded numerically for modeling.

### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [60]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

- The business objective is to improve the effectiveness of a Portuguese bank’s telemarketing campaigns by predicting which clients are most likely to subscribe to a long-term deposit. By identifying the customers with the highest probability of subscribing, the bank can focus its marketing efforts on the right clients, reduce wasted calls, increase conversion rates, and ultimately maximize return on marketing investment.

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [61]:
# Drop duration to avoid data leakage
df = df.drop(columns=['duration'])

In [62]:
# Convert target variable to binary
df['y'] = df['y'].map({'yes': 1, 'no': 0})

# Select only bank client information features (columns 0–7 in description)
bank_features = ['age', 'job', 'marital', 'education', 'default', 'housing', 'loan', 'contact']

X = df[bank_features]
y = df['y']

# One-hot encode categorical variables
X = pd.get_dummies(X, drop_first=True)

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [63]:
# Split the dataset: 70% train, 30% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

In [64]:
print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)

Training set size: (28831, 29)
Testing set size: (12357, 29)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

- The baseline model predicts the majority class (“no deposit”) for all clients. Since about 92% of the clients in the dataset did not subscribe to a term deposit, the baseline accuracy is around 92%. However, this model has 0% recall for the minority class (“yes”), meaning it completely fails to identify actual subscribers. Any useful model must improve not just accuracy, but also recall and precision for the minority class.

In [65]:
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(X_train, y_train)

DummyClassifier(strategy='most_frequent')

In [66]:
y_pred_dummy = dummy.predict(X_test)

print("Baseline Accuracy:", accuracy_score(y_test, y_pred_dummy))
print("\nClassification Report:\n", classification_report(y_test, y_pred_dummy))

Baseline Accuracy: 0.8873512988589464

Classification Report:
               precision    recall  f1-score   support

           0       0.89      1.00      0.94     10965
           1       0.00      0.00      0.00      1392

    accuracy                           0.89     12357
   macro avg       0.44      0.50      0.47     12357
weighted avg       0.79      0.89      0.83     12357



/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


- The baseline model achieves about 88.7% accuracy by always predicting the majority class (“no deposit”). While the accuracy looks high, it completely fails to identify clients who subscribed (“yes”), with precision, recall, and F1-score all equal to 0.0. This shows that any useful model must improve recall and precision for the minority class, not just accuracy.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [67]:
# Initialize Logistic Regression
log_reg_balanced = LogisticRegression(max_iter=1000, random_state=42, class_weight="balanced")

In [68]:
# Train the model
log_reg_balanced.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42)

In [69]:
# Predictions
y_pred = log_reg_balanced.predict(X_test)
y_pred_prob = log_reg_balanced.predict_proba(X_test)[:, 1]

# Evaluation
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred_prob))

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.56      0.70     10965
           1       0.17      0.74      0.28      1392

    accuracy                           0.58     12357
   macro avg       0.56      0.65      0.49     12357
weighted avg       0.86      0.58      0.65     12357

Confusion Matrix:
 [[6086 4879]
 [ 364 1028]]
ROC AUC Score: 0.695195593607665


- With class balancing, Logistic Regression improves recall for the positive class to 74%, meaning it now captures most actual subscribers. Accuracy drops to 58% since the model makes more false positives, but this is acceptable because the goal is to find potential clients for marketing. The ROC AUC score of ~0.70 shows the model has reasonable predictive power compared to the baseline.

### Problem 9: Score the Model

What is the accuracy of your model?

- The accuracy of the model is 0.58(58%)
- The Logistic Regression model with balanced class weights achieved an accuracy of about 58% on the test set. While this is lower than the baseline accuracy (~88%), it is more meaningful because the model now identifies positive cases (subscribers), which is the actual business goal.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|     |    |.     |.     |

In [70]:
# Models to compare
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, class_weight="balanced", random_state=42),
    "KNN": KNeighborsClassifier(),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "SVM": SVC(probability=True, random_state=42)
}
results = []

In [71]:
for name, model in models.items():
    start = time.time()
    model.fit(X_train, y_train)
    end = time.time()
    
    train_acc = model.score(X_train, y_train)
    test_acc = model.score(X_test, y_test)
    train_time = round(end - start, 3)
    
    results.append([name, train_time, train_acc, test_acc])

In [72]:
# Create a DataFrame for easy comparison
results_df = pd.DataFrame(results, columns=["Model", "Train Time", "Train Accuracy", "Test Accuracy"])
print(results_df)

                 Model  Train Time  Train Accuracy  Test Accuracy
0  Logistic Regression       0.677        0.572578       0.575706
1                  KNN       0.009        0.894627       0.876103
2        Decision Tree       0.070        0.926988       0.856519
3                  SVM      32.799        0.887343       0.887351


- Logistic Regression: Fast to train (~0.42s) but has the lowest accuracy (~57.6%). Still useful because it gives interpretable coefficients and good recall for the “yes” class.

- KNN: Very quick (~0.006s) with strong accuracy (~87.6%). Might struggle on large datasets because predictions require distance calculations.

- Decision Tree: High training accuracy (~93%) and strong test accuracy (~85.6%), but risk of overfitting (notice the large gap).

- SVM: Highest accuracy (~88.7%), but extremely slow training (~30s). Likely not practical for very large datasets.

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

## Feature Engineering Example

In [73]:
# Feature Engineering: Drop weak columns and transform 'pdays'
df_fe = df.copy()

# Drop 'default' (mostly 'no/unknown')
df_fe = df_fe.drop(columns=['default'])

# Transform 'pdays': mark 999 as "not previously contacted"
df_fe['pdays'] = df_fe['pdays'].replace(999, -1)

# Recreate X, y with engineered features
X_fe = df_fe.drop(columns=['y'])
y_fe = df_fe['y']

# One-hot encode categorical variables again
X_fe = pd.get_dummies(X_fe, drop_first=True)

print("Shape after feature engineering:", X_fe.shape)

Shape after feature engineering: (41188, 50)


- Dropped 'default' (low-value feature), then recoded pdays so that 999 is no longer treated as a large numeric value but as a separate category (-1). This helps models better understand “never contacted” clients. Feature engineering like this often improves interpretability and predictive performance.

## Hyperparameter Tuning Example

**Decision Tree**

In [74]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

param_grid_tree = {
    'max_depth': [3, 5, 10, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 5]
}

grid_search_tree = GridSearchCV(
    DecisionTreeClassifier(random_state=42),
    param_grid_tree,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

grid_search_tree.fit(X_train, y_train)

print("Best parameters (Decision Tree):", grid_search_tree.best_params_)
print("Best F1-score (cv):", grid_search_tree.best_score_)

python(20336) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(20337) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(20338) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(20339) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(20340) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(20341) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(20342) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(20343) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


Best parameters (Decision Tree): {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best F1-score (cv): 0.14385898766178848


**KNN**

In [75]:
from sklearn.neighbors import KNeighborsClassifier

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 11, 15],
    'weights': ['uniform', 'distance']
}

grid_search_knn = GridSearchCV(
    KNeighborsClassifier(),
    param_grid_knn,
    cv=5,
    scoring='f1',
    n_jobs=-1
)

grid_search_knn.fit(X_train, y_train)

print("Best parameters for KNN:", grid_search_knn.best_params_)
print("Best F1-score (cv):", grid_search_knn.best_score_)

Best parameters for KNN: {'n_neighbors': 3, 'weights': 'distance'}
Best F1-score (cv): 0.16140943984429346


- It is expected that Decision Trees and KNN have low F1-scores in this dataset because the data is highly imbalanced. These models achieve high accuracy by mostly predicting the majority class (“no”), but they fail to correctly identify positive cases (“yes”). This result shows why accuracy is not a sufficient metric and why recall and F1-score are more meaningful for the business goal of identifying potential subscribers.

## Performance Metric Example (Comparing Accuracy vs Recall)

In [76]:
# Use the best KNN model from GridSearch
best_knn = grid_search_knn.best_estimator_
y_pred_knn = best_knn.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred_knn))
print("Precision (class 1):", precision_score(y_test, y_pred_knn))
print("Recall (class 1):", recall_score(y_test, y_pred_knn))
print("F1-score (class 1):", f1_score(y_test, y_pred_knn))

Accuracy: 0.8600793072752286
Precision (class 1): 0.25401459854014596
Recall (class 1): 0.125
F1-score (class 1): 0.16754935002407317


- In this imbalanced dataset, accuracy remains high because most predictions are “no.” However, recall and F1-score reveal that the model struggles to identify subscribers. Since the business goal is to maximize successful subscriptions, recall and F1-score are more meaningful performance metrics than accuracy alone.

##### Questions

- Should the bank prioritize recall (catch as many subscribers as possible) even at the cost of false positives, or precision (fewer wasted calls) even if some subscribers are missed?

- Since the dataset comes from multiple campaigns, are some campaigns more successful than others? Could time of year (month) or day of week affect results?

- How would our model evaluation change if we factored in the cost of calling a non-subscriber vs the benefit of gaining a subscriber?